In [ ]:
%matplotlib                    # Pyplot 그래프 새창에서 열기 옵션

In [ ]:
%matplotlib inline             # Pyplot 그래프 새창에서 열기 '취소' 옵션

In [2]:
pd.set_option('display.max_rows',    100)      # DataFrame 데이터 확인 시 최대 표시 행 수
pd.set_option('display.max_columns', 100)      # DataFrame 데이터 확인 시 최대 표시 열 수

## 라이브러리 import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## 데이터 (최종 선택된 특징데이터) 불러오기

In [4]:
FeatureSelected = pd.read_csv('./ProcessedData/FeatureSelected', sep=',', header=None)
FeatureSelected = pd.DataFrame(np.transpose(FeatureSelected))
FeatureSelected.shape

(360, 5)

## 학습/검증 데이터와 평가 데이터 분할

Fold별 데이터 분할 용이하도록 데이터 Reshape

In [5]:
NoOfData   = int(FeatureSelected.shape[0]/2)   # 데이터 개수 (정상/고장 각각)
FeatNo     = int(FeatureSelected.shape[1])     # 데이터 특징 수 (=데이터 차원)

print(NoOfData, FeatNo)

180 5


In [6]:
NormalSet   = np.array(FeatureSelected.iloc[:NoOfData , :])
AbnormalSet = np.array(FeatureSelected.iloc[NoOfData: , :])
FeatureSelected_Reshaped = pd.DataFrame(np.concatenate([NormalSet , AbnormalSet] , axis=1))
FeatureSelected_Reshaped.shape

(180, 10)

학습/검증 데이터를 160개, 평가 데이터를 20개로 분할

In [7]:
Train_Valid_Data =  FeatureSelected_Reshaped.iloc[:-20, :]
temp_Test_Data = FeatureSelected_Reshaped.iloc[-20:, :]

print(Train_Valid_Data.shape, temp_Test_Data.shape)

(160, 10) (20, 10)


In [8]:
Test_Data = pd.DataFrame(np.concatenate([temp_Test_Data.iloc[:,:FeatNo], temp_Test_Data.iloc[:,FeatNo:]], axis = 0))
Test_Data.shape

(40, 5)

## K값에 맞춰 Fold별 학습/검증 데이터 분할

K값(Fold개수), 데이터 개수(정상/고장 각각), 특징 수(=데이터 차원) 정의하여 '1개 Fold의 (검증)데이터 개수' 계산

In [9]:
NoOfTrainData = Train_Valid_Data.shape[0]

Fold       = 5                                 # Fold 개수 선정 : 데이터 개수의 약수여야 함.
FoldDataNo = int(NoOfTrainData/Fold)           # 1개 Fold 당 (검증)데이터 개수
print(FoldDataNo)

32


Fold별 데이터 분할 (정상/고장 데이터 동일하게 분포하도록)

In [10]:
# Validation Data set
for i in range(Fold):
    
    temp_Valid_Normal   = Train_Valid_Data.iloc[FoldDataNo*i : FoldDataNo*(i+1), :FeatNo]
    temp_Valid_Abnormal = Train_Valid_Data.iloc[FoldDataNo*i : FoldDataNo*(i+1), FeatNo:]
    temp_Valid = pd.DataFrame(np.concatenate([temp_Valid_Normal, temp_Valid_Abnormal] , axis=0))
    
    s = 'Validation_Fold%d = temp_Valid'%(i+1)
    exec(s)

    
# Training Data set
for i in range(Fold):
    
    temp_Train_Front = Train_Valid_Data.iloc[:FoldDataNo*i, :]
    temp_Train_Back  = Train_Valid_Data.iloc[FoldDataNo*(i+1):, :]
    temp_Train_Total = np.concatenate([temp_Train_Front , temp_Train_Back] , axis=0)
    temp_Train_Final = pd.DataFrame(np.concatenate([temp_Train_Total[:, :FeatNo],temp_Train_Total[:, FeatNo:]] , axis=0))
    
    s ='Training_Fold%d  = temp_Train_Final'%(i+1)
    exec(s)


데이터 분할 결과(shape) 확인

In [11]:
Validation_Fold1.shape, Training_Fold1.shape

((64, 5), (256, 5))

## Fold 별 학습/검증 데이터, 평가 데이터 개수에 맞춰 Labeling

Fold별 학습/검증 데이터 및 평가 데이터 개수 확인

In [12]:
NoOfLabel_Train = int(Training_Fold1.shape[0]/2)
NoOfLabel_Valid = int(Validation_Fold1.shape[0]/2)
NoOfLabel_Test  = int(Test_Data.shape[0]/2)

print(NoOfLabel_Train)
print(NoOfLabel_Valid)
print(NoOfLabel_Test)

128
32
20


#### (1) KNN & SVM 레이블 - 정상: 0 // 고장: 1

In [13]:
TrainingFold_Label   = np.zeros(2*NoOfLabel_Train , dtype=int)
ValidationFold_Label = np.zeros(2*NoOfLabel_Valid , dtype=int)
Test_Label           = np.zeros(2*NoOfLabel_Test  , dtype=int)

# 고장데이터(학습용) Label 값 = 1
TrainingFold_Label[NoOfLabel_Train:] = 1

# 고장데이터(검증용) Label 값 = 1
ValidationFold_Label[NoOfLabel_Valid:] = 1

# 고장데이터(평가용) Label 값 = 1
Test_Label[NoOfLabel_Test:] = 1

TrainingFold_Label   = pd.Series(TrainingFold_Label)
ValidationFold_Label = pd.Series(ValidationFold_Label)
Test_Label           = pd.Series(Test_Label)

#TrainingFold_Label
ValidationFold_Label

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    1
33    1
34    1
35    1
36    1
37    1
38    1
39    1
40    1
41    1
42    1
43    1
44    1
45    1
46    1
47    1
48    1
49    1
50    1
51    1
52    1
53    1
54    1
55    1
56    1
57    1
58    1
59    1
60    1
61    1
62    1
63    1
dtype: int32

#### (2) ANN 레이블 (One-hot encoding) - 정상: [1, 0] // 고장: [0, 1]

In [14]:
TrainingFold_Label_forANN   = np.zeros((NoOfLabel_Train*2,2) , dtype=int)
ValidationFold_Label_forANN = np.zeros((NoOfLabel_Valid*2,2) , dtype=int)
Test_Label_forANN           = np.zeros((NoOfLabel_Test *2,2) , dtype=int)

TrainingFold_Label_forANN[:NoOfLabel_Train , 0] = 1    # 정상(학습)데이터 레이블 : [1,0]
TrainingFold_Label_forANN[NoOfLabel_Train: , 1] = 1    # 고장(학습)데이터 레이블 : [0,1]

ValidationFold_Label_forANN[:NoOfLabel_Valid , 0] = 1  # 정상(검증)데이터 레이블 : [1,0]
ValidationFold_Label_forANN[NoOfLabel_Valid: , 1] = 1  # 고장(검증)데이터 레이블 : [0,1]

Test_Label_forANN[:NoOfLabel_Test , 0] = 1
Test_Label_forANN[NoOfLabel_Test :, 1] = 1

TrainingFold_Label_forANN   = pd.DataFrame(TrainingFold_Label_forANN)
ValidationFold_Label_forANN = pd.DataFrame(ValidationFold_Label_forANN)
Test_Label_forANN           = pd.DataFrame(Test_Label_forANN)

Test_Label_forANN
#ValidationFold_Label_forANN

,0,1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


## k-fold 데이터 및 레이블 저장

k-fold 데이터 (Training & Validation) 저장

In [15]:
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'  %(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    
    c1 = 'Training_Fold%d.to_csv(  path1, sep = ",", header = None, index = None)'%(i+1)
    c2 = 'Validation_Fold%d.to_csv(path2, sep = ",", header = None, index = None)'%(i+1)
    exec(c1)
    exec(c2)
    
Test_Data.to_csv('./K_FoldData/Test_Data', sep = ",", header = None, index = None)

레이블 (Training & Validation) 저장

In [16]:
TrainingFold_Label.to_csv(  './K_FoldData/TrainingFold_Label', header = None, index = None)
ValidationFold_Label.to_csv('./K_FoldData/ValidationFold_Label', header = None, index = None)
Test_Label.to_csv(          './K_FoldData/Test_Label', header = None, index = None)

TrainingFold_Label_forANN.to_csv(  './K_FoldData/TrainingFold_Label_forANN', sep = ",", header = None, index = None)
ValidationFold_Label_forANN.to_csv('./K_FoldData/ValidationFold_Label_forANN', sep = ",", header = None, index = None)
Test_Label_forANN.to_csv(          './K_FoldData/Test_Label_forANN', sep = ",", header = None, index = None)

C:\Users\erich\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\erich\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
C:\Users\erich\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


## 최종 AI모델 학습용 전체 데이터 및 레이블 저장

전체 데이터 저장

In [17]:
Normal_Training_All   = Train_Valid_Data.iloc[:,:FeatNo]
Abnormal_Training_All = Train_Valid_Data.iloc[:,FeatNo:]

In [18]:
Normal_Training_All.shape, Abnormal_Training_All.shape

((160, 5), (160, 5))

In [19]:
Training_All = pd.DataFrame(np.concatenate([Normal_Training_All, Abnormal_Training_All], axis = 0))
Training_All.shape

(320, 5)

#### 전체 데이터 레이블 (1) SVM 및 KNN 맞춤형 (Label encoding)

In [20]:
Training_All_Label = np.zeros(NoOfTrainData*2)

Training_All_Label[NoOfTrainData:] = 1    # 고장데이터(학습용) Label 값 = 1
Training_All_Label = pd.Series(Training_All_Label)

Training_All_Label.shape

(320,)

#### 전체 데이터 레이블 (2) ANN 맞춤형 (One-hot encoding)

In [21]:
Training_All_Label_forANN = np.zeros((NoOfTrainData*2,2))

Training_All_Label_forANN[:NoOfTrainData , 0] = 1
Training_All_Label_forANN[NoOfTrainData: , 1] = 1
Training_All_Label_forANN = pd.DataFrame(Training_All_Label_forANN)

Training_All_Label_forANN.shape

(320, 2)

전체 데이터 & 레이블 저장

In [22]:
Training_All.to_csv('./K_FoldData/Training_All', sep = ",", header = None, index = None)
Training_All_Label.to_csv('./K_FoldData/Training_All_Label', sep = ",", header = None, index = None)
Training_All_Label_forANN.to_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None, index = None)

C:\Users\erich\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
